In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from Latent_Dirichlet_Allocation import LDA_In_EM

%matplotlib inline

In [2]:
df_tweet = pd.read_csv('csv_data/20250127_tweet_data.csv')
df_tweet['text'] = df_tweet['text'].str.removeprefix('[').str.removesuffix(']')
df_tweet = df_tweet[['text']]

In [ ]:
model = LDA_In_EM(df_tweet, topic_num=5)
model.fit()

print(f'topic_θ: {model.topic_θ}')
print(f'word_Φ:  {model.word_Φ}')

学習回数：0


In [4]:
pd_θ, pd_Φ = model.stats_info()

In [6]:
pd_θ.to_parquet('./parquet_data/20250227_tweet_EM_θ.parquet')
pd_Φ.to_parquet('./parquet_data/20250227_tweet_EM_Φ.parquet')